# Structured Streaming using the Python DataFrames API

Apache Spark includes a high-level stream processing API, [Structured Streaming](http://spark.apache.org/docs/latest/structured-streaming-programming-guide.html). In this notebook we take a quick look at how to use the DataFrame API to build Structured Streaming applications. We want to compute real-time metrics like running counts and windowed counts on a stream of timestamped actions (e.g. Open, Close, etc).

To run this notebook, import it and attach it to a Spark cluster.

## Sample Data
We have some sample action data as files in `/databricks-datasets/structured-streaming/events/` which we are going to use to build this appication. Let's take a look at the contents of this directory.

In [3]:
# Look at the content of the following folder: /databricks-datasets/structured-streaming/events/
# What do you see?
display(dbutils.fs.ls('dbfs:/databricks-datasets/structured-streaming/events/'))

path,name,size
dbfs:/databricks-datasets/structured-streaming/events/file-0.json,file-0.json,72530
dbfs:/databricks-datasets/structured-streaming/events/file-1.json,file-1.json,72961
dbfs:/databricks-datasets/structured-streaming/events/file-10.json,file-10.json,73025
dbfs:/databricks-datasets/structured-streaming/events/file-11.json,file-11.json,72999
dbfs:/databricks-datasets/structured-streaming/events/file-12.json,file-12.json,72987
dbfs:/databricks-datasets/structured-streaming/events/file-13.json,file-13.json,73006
dbfs:/databricks-datasets/structured-streaming/events/file-14.json,file-14.json,73003
dbfs:/databricks-datasets/structured-streaming/events/file-15.json,file-15.json,73007
dbfs:/databricks-datasets/structured-streaming/events/file-16.json,file-16.json,72978
dbfs:/databricks-datasets/structured-streaming/events/file-17.json,file-17.json,73008


There are about 50 JSON files in the directory. Let's see what each JSON file contains.

In [5]:
# Look at the function head in dbutils
# Open one file
dbutils.fs.head("dbfs:/databricks-datasets/structured-streaming/events/file-0.json",1000).split()

[Truncated to first 1000 bytes]
Out[56]: ['{"time":1469501107,"action":"Open"}',
 '{"time":1469501147,"action":"Open"}',
 '{"time":1469501202,"action":"Open"}',
 '{"time":1469501219,"action":"Open"}',
 '{"time":1469501225,"action":"Open"}',
 '{"time":1469501234,"action":"Open"}',
 '{"time":1469501245,"action":"Open"}',
 '{"time":1469501246,"action":"Open"}',
 '{"time":1469501248,"action":"Open"}',
 '{"time":1469501256,"action":"Open"}',
 '{"time":1469501264,"action":"Open"}',
 '{"time":1469501266,"action":"Open"}',
 '{"time":1469501267,"action":"Open"}',
 '{"time":1469501269,"action":"Open"}',
 '{"time":1469501271,"action":"Open"}',
 '{"time":1469501282,"action":"Open"}',
 '{"time":1469501285,"action":"Open"}',
 '{"time":1469501291,"action":"Open"}',
 '{"time":1469501297,"action":"Open"}',
 '{"time":1469501303,"action":"Open"}',
 '{"time":1469501322,"action":"Open"}',
 '{"time":1469501335,"action":"Open"}',
 '{"time":1469501344,"action":"Open"}',
 '{"time":1469501346,"action":"Open"}',
 '{"time":1469501349,"action":"Open"}',
 '{"time":1469501357,"action":"Open"}',
 '{"time":1469501366,"action":"Open"}',
 '{"time":1469501371,"action":']

Each line in the file contains JSON record with two fields - `time` and `action`. Let's try to analyze these files interactively.

## Batch/Interactive Processing
The usual first step in attempting to process the data is to interactively query the data. Let's define a static DataFrame on the files, and give it a table name.

In [8]:
from pyspark.sql.types import *

inputPath = "/databricks-datasets/structured-streaming/events/"

# Since we know the data format already, let's define the schema to speed up processing (no need for Spark to infer schema)
jsonSchema = StructType(
  [ StructField("time", TimestampType(), True),
   StructField("action", StringType(), True) ]
)



In [9]:
# Read one json file taking into account the defined schema and display the content 
jsonfile_schema = spark.read.schema(jsonSchema).json(imputPath)
jsonfile_schema.printSchema()
jsonfile_schema.show()

root
-- time: timestamp (nullable = true)
-- action: string (nullable = true)

+-------------------+------+
 time|action|
+-------------------+------+
2016-07-26 02:45:07| Open|
2016-07-26 02:45:47| Open|
2016-07-26 02:46:42| Open|
2016-07-26 02:46:59| Open|
2016-07-26 02:47:05| Open|
2016-07-26 02:47:14| Open|
2016-07-26 02:47:25| Open|
2016-07-26 02:47:26| Open|
2016-07-26 02:47:28| Open|
2016-07-26 02:47:36| Open|
2016-07-26 02:47:44| Open|
2016-07-26 02:47:46| Open|
2016-07-26 02:47:47| Open|
2016-07-26 02:47:49| Open|
2016-07-26 02:47:51| Open|
2016-07-26 02:48:02| Open|
2016-07-26 02:48:05| Open|
2016-07-26 02:48:11| Open|
2016-07-26 02:48:17| Open|
2016-07-26 02:48:23| Open|
+-------------------+------+
only showing top 20 rows

- Compare the dates from the output without schema and with it. 
- Did you notice that inputPath is a folder?

In [11]:
# Calculate the total number of 'Open' and 'Close' actions 
jsonfile_schema.groupBy('action').count().show()

+------+-----+
action|count|
+------+-----+
 Open| 1469|
 Close| 531|
+------+-----+

In [12]:
# Determine min and max date
from pyspark.sql.functions import min, max

min_date = jsonfile_schema.select(min('time')).show()
max_date = jsonfile_schema.select(max('time')).show()

+-------------------+
 min(time)|
+-------------------+
2016-07-26 02:45:07|
+-------------------+

+-------------------+
 max(time)|
+-------------------+
2016-07-26 04:17:11|
+-------------------+

In [13]:
# Calculate the number of "open" and "close" actions with one hour windows: staticCountsDF
# Look at groupBy(..., window) function
from pyspark.sql.functions import window

staticCountsDF = jsonfile_schema.groupBy('action', window('time', '1 hour')).count()
staticCountsDF.display()

action,window,count
Close,"List(2016-07-26T04:00:00.000+0000, 2016-07-26T05:00:00.000+0000)",176
Open,"List(2016-07-26T04:00:00.000+0000, 2016-07-26T05:00:00.000+0000)",289
Close,"List(2016-07-26T03:00:00.000+0000, 2016-07-26T04:00:00.000+0000)",344
Open,"List(2016-07-26T03:00:00.000+0000, 2016-07-26T04:00:00.000+0000)",1001
Open,"List(2016-07-26T02:00:00.000+0000, 2016-07-26T03:00:00.000+0000)",179
Close,"List(2016-07-26T02:00:00.000+0000, 2016-07-26T03:00:00.000+0000)",11


In [14]:
# Make this window a sliding window (30 minutes overlap): staticCountsSW
staticCountsSW = jsonfile_schema.groupBy('action', window('time', '1 hour', '30 minutes')).count()
staticCountsSW.display()
# => Certains data will belong to 2 windows

action,window,count
Close,"List(2016-07-26T02:30:00.000+0000, 2016-07-26T03:30:00.000+0000)",116
Close,"List(2016-07-26T03:30:00.000+0000, 2016-07-26T04:30:00.000+0000)",415
Close,"List(2016-07-26T04:00:00.000+0000, 2016-07-26T05:00:00.000+0000)",176
Open,"List(2016-07-26T03:30:00.000+0000, 2016-07-26T04:30:00.000+0000)",789
Open,"List(2016-07-26T04:00:00.000+0000, 2016-07-26T05:00:00.000+0000)",289
Close,"List(2016-07-26T03:00:00.000+0000, 2016-07-26T04:00:00.000+0000)",344
Open,"List(2016-07-26T03:00:00.000+0000, 2016-07-26T04:00:00.000+0000)",1001
Open,"List(2016-07-26T02:30:00.000+0000, 2016-07-26T03:30:00.000+0000)",680
Open,"List(2016-07-26T02:00:00.000+0000, 2016-07-26T03:00:00.000+0000)",179
Close,"List(2016-07-26T02:00:00.000+0000, 2016-07-26T03:00:00.000+0000)",11


In [15]:
# Register staticCountsDF (createOrReplaceTempView) as table 'static_counts'
staticCountsDF.createOrReplaceTempView('static_counts')

Now we can directly use SQL to query the table.

In [17]:
%sql
-- Count all Open and Close actions in the table static_counts  
select action, sum(count) as total_actions from static_counts group by action


action,total_actions
Close,531
Open,1469


In [18]:
%sql
-- How many actions (Close and Open separately) is within each time window (in the table static_counts)
-- Make a plot
select action, date_format(window.end, "MMM-dd HH:mm") as time, count from static_counts order by time, action


action,time,count
Close,Jul-26 03:00,11
Open,Jul-26 03:00,179
Close,Jul-26 04:00,344
Open,Jul-26 04:00,1001
Close,Jul-26 05:00,176
Open,Jul-26 05:00,289


Note the two ends of the graph. The close actions are generated such that they are after the corresponding open actions, so there are more "opens" in the beginning and more "closes" in the end.

## Stream Processing 
Now that we have analyzed the data interactively, let's convert this to a streaming query that continuously updates as data comes. Since we just have a static set of files, we are going to emulate a stream from them by reading one file at a time, in the chronological order they were created. The query we have to write is pretty much the same as the interactive query above.

In [21]:
from pyspark.sql.functions import *

# Read data from a file
# Similar to definition of staticInputDF above, just using `readStream` instead of `read`
streamingInputDF = (
  spark
    .readStream                       
    .schema(jsonSchema)               # Set the schema of the JSON data
    .option("maxFilesPerTrigger", 1)  # Treat a sequence of files as a stream by picking one file at a time
    .json(inputPath)
)

# Do some transformations
# Same query as staticInputDF
streamingCountsDF = (                 
  streamingInputDF
    .groupBy(
      streamingInputDF.action, 
      window(streamingInputDF.time, "1 hour"))
    .count()
)

# Is this DF actually a streaming DF?
streamingCountsDF.isStreaming

Out[64]: True

In [22]:
streamingInputDF.display()

time,action
2016-07-26T02:45:07.000+0000,Open
2016-07-26T02:45:47.000+0000,Open
2016-07-26T02:46:42.000+0000,Open
2016-07-26T02:46:59.000+0000,Open
2016-07-26T02:47:05.000+0000,Open
2016-07-26T02:47:14.000+0000,Open
2016-07-26T02:47:25.000+0000,Open
2016-07-26T02:47:26.000+0000,Open
2016-07-26T02:47:28.000+0000,Open
2016-07-26T02:47:36.000+0000,Open


In [23]:
streamingCountsDF.display()

action,window,count
Close,"List(2016-07-26T13:00:00.000+0000, 2016-07-26T14:00:00.000+0000)",1028
Open,"List(2016-07-26T05:00:00.000+0000, 2016-07-26T06:00:00.000+0000)",1000
Open,"List(2016-07-26T11:00:00.000+0000, 2016-07-26T12:00:00.000+0000)",991
Close,"List(2016-07-26T06:00:00.000+0000, 2016-07-26T07:00:00.000+0000)",1011
Open,"List(2016-07-26T10:00:00.000+0000, 2016-07-26T11:00:00.000+0000)",1007
Close,"List(2016-07-26T04:00:00.000+0000, 2016-07-26T05:00:00.000+0000)",815
Close,"List(2016-07-26T12:00:00.000+0000, 2016-07-26T13:00:00.000+0000)",960
Close,"List(2016-07-26T08:00:00.000+0000, 2016-07-26T09:00:00.000+0000)",985
Close,"List(2016-07-26T11:00:00.000+0000, 2016-07-26T12:00:00.000+0000)",1028
Open,"List(2016-07-26T13:00:00.000+0000, 2016-07-26T14:00:00.000+0000)",1006


As you can see, `streamingCountsDF` is a streaming Dataframe (`streamingCountsDF.isStreaming` was `true`). You can start streaming computation, by defining the sink and starting it. 
In our case, we want to interactively query the counts (same queries as above), so we will set the complete set of 1 hour counts to be in a in-memory table.

In [25]:
spark.conf.set("spark.sql.shuffle.partitions", "2")  # keep the size of shuffles small

query = (
  streamingCountsDF
    .writeStream
    .format("memory")        # memory = store in-memory table 
    .queryName("counts")     # counts = name of the in-memory table
    .outputMode("complete")  # complete = all the counts should be in the table
    #.trigger(processingTime='15 seconds')
    .start()
)

`query` is a handle to the streaming query that is running in the background. This query is continuously picking up files and updating the windowed counts. 

Note the status of query in the above cell. The progress bar shows that the query is active. 
Furthermore, if you expand the `> counts` above, you will find the number of files they have already processed. 

Let's wait a bit for a few files to be processed and then interactively query the in-memory `counts` table.

In [27]:
%sql
SELECT *
FROM counts

action,window,count
Open,"List(2016-07-26T02:00:00.000+0000, 2016-07-26T03:00:00.000+0000)",179
Close,"List(2016-07-26T04:00:00.000+0000, 2016-07-26T05:00:00.000+0000)",176
Close,"List(2016-07-26T02:00:00.000+0000, 2016-07-26T03:00:00.000+0000)",11
Open,"List(2016-07-26T03:00:00.000+0000, 2016-07-26T04:00:00.000+0000)",1001
Close,"List(2016-07-26T03:00:00.000+0000, 2016-07-26T04:00:00.000+0000)",344
Open,"List(2016-07-26T04:00:00.000+0000, 2016-07-26T05:00:00.000+0000)",289


In [28]:
from time import sleep
sleep(5)  # wait a bit for computation to start

In [29]:
%sql
select action, date_format(window.end, "MMM-dd HH:mm") as time, count
from counts
order by time, action

action,time,count
Close,Jul-26 03:00,11
Open,Jul-26 03:00,179
Close,Jul-26 04:00,344
Open,Jul-26 04:00,1001
Close,Jul-26 05:00,815
Open,Jul-26 05:00,999
Close,Jul-26 06:00,1003
Open,Jul-26 06:00,1000
Close,Jul-26 07:00,1011
Open,Jul-26 07:00,993


We see the timeline of windowed counts (similar to the static one earlier) building up. If we keep running this interactive query repeatedly, we will see the latest updated counts which the streaming query is updating in the background.

In [31]:
sleep(5)  # wait a bit more for more data to be computed

In [32]:
%sql select action, date_format(window.end, "MMM-dd HH:mm") as time, count from counts order by time, action

action,time,count
Close,Jul-26 03:00,11
Open,Jul-26 03:00,179
Close,Jul-26 04:00,344
Open,Jul-26 04:00,1001
Close,Jul-26 05:00,815
Open,Jul-26 05:00,999
Close,Jul-26 06:00,1003
Open,Jul-26 06:00,1000
Close,Jul-26 07:00,1011
Open,Jul-26 07:00,993


In [33]:
sleep(5)  # wait a bit more for more data to be computed

In [34]:
%sql select action, date_format(window.end, "MMM-dd HH:mm") as time, count from counts order by time, action

action,time,count
Close,Jul-26 03:00,11
Open,Jul-26 03:00,179
Close,Jul-26 04:00,344
Open,Jul-26 04:00,1001
Close,Jul-26 05:00,815
Open,Jul-26 05:00,999
Close,Jul-26 06:00,1003
Open,Jul-26 06:00,1000
Close,Jul-26 07:00,1011
Open,Jul-26 07:00,993


Also, let's see the total number of "opens" and "closes".

In [36]:
%sql 
select action, sum(count) as total_count 
from counts 
group by action 
order by action

action,total_count
Close,42499
Open,43501


If you keep running the above query repeatedly, you will always find that the number of "opens" is more than the number of "closes", as expected in a data stream where a "close" always appear after corresponding "open". This shows that Structured Streaming ensures **prefix integrity**. Read the blog posts linked below if you want to know more.

Note that there are only a few files, so consuming all of them there will be no updates to the counts. Rerun the query if you want to interact with the streaming query again.

Finally, you can stop the query running in the background, either by clicking on the 'Cancel' link in the cell of the query, or by executing `query.stop()`. Either way, when the query is stopped, the status of the corresponding cell above will automatically update to `TERMINATED`.

In [38]:
query.stop()